## Pre

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
from google.cloud import bigquery
import os
import warnings

!pip install openpyxl ## read excel files
!pip install wrds
import wrds

from datetime import timedelta

# pd.set_option('display.max_columns', None)

os.chdir('/home/jupyter')
!pwd
# # %cd 
# !pwd
# (base) jupyter@starfire-1:~$ pip install jupyter-console
# (base) jupyter@starfire-1:~$ !jupyter console --existing
%connect_info


/home/jupyter
{
  "shell_port": 42931,
  "iopub_port": 33439,
  "stdin_port": 44033,
  "control_port": 42503,
  "hb_port": 51043,
  "ip": "127.0.0.1",
  "key": "0559085d-d2d0a16d116f5a682e868ff7",
  "transport": "tcp",
  "signature_scheme": "hmac-sha256",
  "kernel_name": "conda-root-py",
  "jupyter_session": "/home/jupyter/starfire/notebooks/2.1_retrieve_compu_stock_prc_for_pb_deals.ipynb"
}

Paste the above JSON into a file, and connect with:
    $> jupyter <app> --existing <file>
or, if you are local, you can connect with just:
    $> jupyter <app> --existing kernel-2ec683f2-9912-4768-92ed-89c2681d7206.json
or even just:
    $> jupyter <app> --existing
if this is the most recent Jupyter kernel you have started.


In [2]:
print('----------------\n Establishing WRDS connection:')
# establish connection
conn = wrds.Connection()

# crsp_list = conn.list_tables(library='crsp')
# crsp_list

----------------
 Establishing WRDS connection:


Enter your WRDS username [jupyter]: 
Enter your password: ········


OperationalError: (psycopg2.OperationalError) connection to server at "wrds-pgdata.wharton.upenn.edu" (165.123.60.118), port 9737 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/20/e3q8)

## Download data

In [3]:
## Download data from BQ, Startups with MnA deals

print('downloading pitchbook data... \n')

# Define project on BQ
project_id = "starfire-410116"

# Authenticate to Google Cloud
client = bigquery.Client(project=project_id)

# Set allow_large_results to Truepoti
job_config = bigquery.QueryJobConfig()
job_config.allow_large_results = True

query_txt = """
SELECT 
    targetid, mna_dealid, mna_dealsize, 
    mna_anndate, mna_dealdate, 
    mna_investorid, mna_investorname,
    mna_investorcikcode, mna_investorticker,
    mna_investorexchange, mna_investorparent, mna_investorctry,
FROM `starfire-410116.derived.all_sups` 
WHERE mna_dealid IS NOT NULL
""" 

# Construct the SQL query to fetch data from BigQuery
query = f'{query_txt}'

# Fetch data from BigQuery
query_job = client.query(query, job_config=job_config)
results = query_job.result()  # Waits for the query to complete

# Convert the result to a pandas DataFrame
df = results.to_dataframe()


## Format dates
df['mna_anndate'] = [ts.strftime('%Y-%m-%d') if pd.notna(ts) else np.nan for ts in df['mna_anndate']]
df['mna_dealdate'] = [ts.strftime('%Y-%m-%d') if pd.notna(ts) else np.nan for ts in df['mna_dealdate']]
df['mna_eventdate'] = df['mna_anndate'].combine_first(df['mna_dealdate'])

#standardize cik codes to 10 digits with 00
df['mna_investorcikcode'] = [item.strip().rjust(10, '0') if isinstance(item, str) else item for item in df['mna_investorcikcode']]

print(df.columns)
df

downloading pitchbook data... 

Index(['targetid', 'mna_dealid', 'mna_dealsize', 'mna_anndate', 'mna_dealdate',
       'mna_investorid', 'mna_investorname', 'mna_investorcikcode',
       'mna_investorticker', 'mna_investorexchange', 'mna_investorparent',
       'mna_investorctry', 'mna_eventdate'],
      dtype='object')


,targetid,mna_dealid,mna_dealsize,mna_anndate,mna_dealdate,mna_investorid,mna_investorname,mna_investorcikcode,mna_investorticker,mna_investorexchange,mna_investorparent,mna_investorctry,mna_eventdate
0,52298-56,46282-96T,NaN,NaN,2015-02-18,108237-43,AVL List,None,None,None,Robert Bosch Engineering and Business Solutions,Austria,2015-02-18
1,58110-13,79999-39T,NaN,NaN,2016-12-19,169050-07,Harbour BioMed,None,02142,HKG,None,Hong Kong,2016-12-19
2,93819-79,248895-91T,NaN,NaN,2022-12-05,495749-98,WasteCo Holdings NZ,None,None,None,None,New Zealand,2022-12-05
3,416586-16,136837-54T,NaN,NaN,2020-05-20,185763-25,Binance,None,None,None,None,Cayman Islands,2020-05-20
4,494440-39,191709-55T,NaN,2019-01-02,2019-01-02,176590-99,Sheypoor,None,None,None,None,Iran,2019-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
47055,501471-10,224763-22T,NaN,NaN,2023-02-27,12465-46,Saga Group,None,SAGA,LON,None,United Kingdom,2023-02-27
47056,140341-69,61839-37T,NaN,2015-07-25,2017-06-08,531384-22,Nottingham College,None,None,None,None,United Kingdom,2015-07-25
47057,146547-91,186749-29T,NaN,NaN,2022-01-17,118066-78,MyTutor,None,None,None,None,United Kingdom,2022-01-17
47058,63654-94,34437-43T,NaN,NaN,2012-12-18,63667-81,Regenerys,None,None,None,None,United Kingdom,2012-12-18


In [4]:
df['mna_investorcikcode'] = df['mna_investorcikcode'].str.split(',') 
df = df.explode('mna_investorcikcode').reset_index(drop=True)
df


,targetid,mna_dealid,mna_dealsize,mna_anndate,mna_dealdate,mna_investorid,mna_investorname,mna_investorcikcode,mna_investorticker,mna_investorexchange,mna_investorparent,mna_investorctry,mna_eventdate
0,52298-56,46282-96T,NaN,NaN,2015-02-18,108237-43,AVL List,None,None,None,Robert Bosch Engineering and Business Solutions,Austria,2015-02-18
1,58110-13,79999-39T,NaN,NaN,2016-12-19,169050-07,Harbour BioMed,None,02142,HKG,None,Hong Kong,2016-12-19
2,93819-79,248895-91T,NaN,NaN,2022-12-05,495749-98,WasteCo Holdings NZ,None,None,None,None,New Zealand,2022-12-05
3,416586-16,136837-54T,NaN,NaN,2020-05-20,185763-25,Binance,None,None,None,None,Cayman Islands,2020-05-20
4,494440-39,191709-55T,NaN,2019-01-02,2019-01-02,176590-99,Sheypoor,None,None,None,None,Iran,2019-01-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...
48351,501471-10,224763-22T,NaN,NaN,2023-02-27,12465-46,Saga Group,None,SAGA,LON,None,United Kingdom,2023-02-27
48352,140341-69,61839-37T,NaN,2015-07-25,2017-06-08,531384-22,Nottingham College,None,None,None,None,United Kingdom,2015-07-25
48353,146547-91,186749-29T,NaN,NaN,2022-01-17,118066-78,MyTutor,None,None,None,None,United Kingdom,2022-01-17
48354,63654-94,34437-43T,NaN,NaN,2012-12-18,63667-81,Regenerys,None,None,None,None,United Kingdom,2012-12-18


In [5]:
# Compute statistics
un_targetid = df['targetid'].nunique()
un_mna_dealid = df['mna_dealid'].nunique()
un_mna_investors = df['mna_investorid'].nunique()
# un_mna_dealid_eventdate_non_null = df[df['mna_eventdate'].notnull()]['mna_dealid'].nunique()
un_mna_dealid_eventdate_non_null = df[df['mna_eventdate'].notna()]['mna_dealid'].nunique()
un_mna_dealid_anndate_non_null = df[df['mna_eventdate'].notna()]['mna_dealid'].nunique()
un_mna_dealid_dealdate_non_null = df[df['mna_anndate'].notna()]['mna_dealid'].nunique()
un_mna_dealid_dealsize_non_null = df[df['mna_dealsize'].notnull()]['mna_dealid'].nunique()
un_mna_dealid_dealsize_eventdate_non_null = df[df['mna_dealsize'].notnull() & df['mna_eventdate'].notna()]['mna_dealid'].nunique()
un_mna_dealid_investorcikcode_non_null = df[df['mna_investorcikcode'].notnull()]['mna_dealid'].nunique()
un_mna_dealid_investorticker_non_null = df[df['mna_investorticker'].notnull()]['mna_dealid'].nunique()
un_mna_dealid_dealsize_eventdate_investorcikcode_non_null = df[df['mna_dealsize'].notnull() & df['mna_eventdate'].notna() & df['mna_investorcikcode'].notnull()]['mna_dealid'].nunique()
un_mna_dealid_dealsize_eventdate_investorticker_non_null = df[df['mna_dealsize'].notnull() & df['mna_eventdate'].notna() & df['mna_investorticker'].notnull()]['mna_dealid'].nunique()
un_mna_investorcikcode = df['mna_investorcikcode'].nunique()
un_mna_investorticker = df['mna_investorticker'].nunique()

# Print the results
print(f'Unique targetid: {un_targetid}')
print(f'Unique mna_dealid: {un_mna_dealid}')
print(f'Unique mna_investor: {un_mna_investors}')
print(f'Unique mna_dealid with unique targetid:', df.groupby('targetid')['mna_dealid'].nunique().count())
print(f'Unique mna_dealid with mna_eventdate non null: {un_mna_dealid_eventdate_non_null}')
print(f'Unique mna_dealid with mna_anndate non null: {un_mna_dealid_anndate_non_null}')
print(f'Unique mna_dealid with mna_dealdate non null: {un_mna_dealid_dealdate_non_null}')
print(f'Unique mna_dealid with mna_dealsize non null: {un_mna_dealid_dealsize_non_null}')
print(f'Unique mna_dealid with mna_dealsize non null and mna_eventdate non null: {un_mna_dealid_dealsize_eventdate_non_null}')
print(f'Unique mna_dealid with mna_investorcikcode not null: {un_mna_dealid_investorcikcode_non_null}')
print(f'Unique mna_dealid with mna_investorticker not null: {un_mna_dealid_investorticker_non_null}')
print(f'Unique mna_dealid with mna_dealsize non null and mna_eventdate non null and mna_investorcikcode not null: {un_mna_dealid_dealsize_eventdate_investorcikcode_non_null}')
print(f'Unique mna_dealid with mna_dealsize non null and mna_eventdate non null and mna_investorticker not null: {un_mna_dealid_dealsize_eventdate_investorticker_non_null}')
print(f'Unique mna_investorcikcode: {un_mna_investorcikcode}')
print(f'Unique mna_investorticker: {un_mna_investorticker}')



Unique targetid: 43644
Unique mna_dealid: 46446
Unique mna_investor: 26499
Unique mna_dealid with unique targetid: 43644
Unique mna_dealid with mna_eventdate non null: 45335
Unique mna_dealid with mna_anndate non null: 45335
Unique mna_dealid with mna_dealdate non null: 8362
Unique mna_dealid with mna_dealsize non null: 15304
Unique mna_dealid with mna_dealsize non null and mna_eventdate non null: 15269
Unique mna_dealid with mna_investorcikcode not null: 20309
Unique mna_dealid with mna_investorticker not null: 14869
Unique mna_dealid with mna_dealsize non null and mna_eventdate non null and mna_investorcikcode not null: 9660
Unique mna_dealid with mna_dealsize non null and mna_eventdate non null and mna_investorticker not null: 8271
Unique mna_investorcikcode: 8279
Unique mna_investorticker: 5072


## Compustat stock prices around event date for PB deals, upload to BigQuery

In [6]:
## Check cikcode present in compustat dataset. North America v Global

unique_ciks = df['mna_investorcikcode'].dropna().astype(str).unique().tolist()
unique_ciks = [item.strip().rjust(10, '0') for item in unique_ciks]
print(f'\nLength unique ciks: {len(unique_ciks)}')

# Create single string of type 'yyy', 'yyy', ...  to pass to SQL query
unique_ciks = ', '.join([f"'{item}'" for item in unique_ciks])

# Get list of cik codes in Global Compustat stock price dataset
stock_ciks_glb = conn.raw_sql(
    f"""
        SELECT
            gvkey, cik, conm
        FROM comp_global_daily.g_company
        where cik IN ({unique_ciks})
    """
)
print(f'\nGlobal: \n{stock_ciks_glb}')


# Get list of cik codes in North America Compustat stock price dataset
stock_ciks_na = conn.raw_sql(
    f"""
        SELECT
            gvkey, cik, conm
        FROM comp_na_daily_all.company
        where cik IN ({unique_ciks})
    """
)
print(f'\nNorth America: \n{stock_ciks_na}')


# Concatenate lists to get total unique number of ciks
stock_ciks_all = pd.concat([stock_ciks_na,stock_ciks_glb],axis=0 )
stock_ciks_all = stock_ciks_all.reset_index(drop=True)
print(f'\nAll: \n{stock_ciks_all}')



Length unique ciks: 8279

Global: 
      gvkey         cik                          conm
0    001661  0001163739         NABORS INDUSTRIES LTD
1    001932  0001303523      BRITISH AMER TOBACCO PLC
2    002338  0000013522                     REXAM PLC
3    002410  0000313807                        BP PLC
4    002411  0000756620                  BT GROUP PLC
..      ...         ...                           ...
590  345606  0001978867                   CADELER A/S
591  348615  0001901279                     NAYAX LTD
592  348892  0001828098         STEAKHOLDER FOODS LTD
593  351038  0001671502     QUOIN PHARMACEUTICALS LTD
594  356128  0001985487  JOINT STOCK COMPANY KASPI KZ

[595 rows x 3 columns]

North America: 
       gvkey         cik                          conm
0     001004  0000001750                      AAR CORP
1     001013  0000061478    ADC TELECOMMUNICATIONS INC
2     001056  0000002601                  AEROFLEX INC
3     001072  0000859163                      AVX CORP


In [7]:
# Check how many deals with CIK with stock market data as per last cell
print(len(stock_ciks_all['gvkey'].unique()))
print(len(stock_ciks_all['cik'].unique()))

matching_rows = df[df['mna_investorcikcode'].isin(stock_ciks_all['cik'])]
matching_rows


## Looks like NA includes all but 2 ciks in Global. Why then the diff datasets?
matching_rows = stock_ciks_na[stock_ciks_na['cik'].isin(stock_ciks_glb['cik'])]
matching_rows



4293
4293


,gvkey,cik,conm
31,001661,0001163739,NABORS INDUSTRIES LTD
49,001932,0001303523,BRITISH AMER TOBACCO PLC
72,002338,0000013522,REXAM PLC
76,002410,0000313807,BP PLC
77,002411,0000756620,BT GROUP PLC
...,...,...,...
4280,345606,0001978867,CADELER A/S
4285,348615,0001901279,NAYAX LTD
4286,348892,0001828098,STEAKHOLDER FOODS LTD
4289,351038,0001671502,QUOIN PHARMACEUTICALS LTD


In [22]:
cik = '0000899115'
start_date = '2009-10-14'
end_date = '2009-10-20'

stock_ciks_na = conn.raw_sql(
    f"""
    SELECT
        gvkey, conm, cusip,
        exchg, fic, tpci, iid,
        datadate, cik, prccd, prcstd, cshoc, cshtrd, curcdd
    FROM comp_na_daily_all.secd
    WHERE 
        cik = '0000899115'
limit 10
"""
)

stock_ciks_na = pd.DataFrame(stock_ciks_na)

# Add additional columns to the result (start, end, MnAAnnDate, RIC)
stock_ciks_na['Start'] = start_date
stock_ciks_na['End'] = end_date
stock_ciks_na['mna_investorcikcode'] = cikcode
stock_ciks_na['mna_eventdate'] = mna_eventdate
stock_ciks_na['mna_dealid'] = mnapbdealid
stock_ciks_na['loop_counter'] = counter
# stock_ciks_na = stock_ciks_na.groupby('mna_dealid').apply(calculate_lag).reset_index(drop=True)

stock_ciks_na


,gvkey,conm,cusip,exchg,fic,tpci,iid,datadate,cik,prccd,prcstd,cshoc,cshtrd,curcdd,Start,End,mna_investorcikcode,mna_eventdate,mna_dealid,loop_counter


In [8]:
# Loop to get stock price for each cik
## Create consecutive lags
def calculate_lag(group):
    # Find the index of the row where Date equals MnAAnnDate
    matching_indices = group.index[group['datadate'] == group['mna_eventdate']].tolist()
    
    if matching_indices:
        # Exact match found
        baseline_index = matching_indices[0]
    else:
        # No exact match, find the first date after MnAAnnDate
        after_mna_date = group[group['datadate'] > group['mna_eventdate'].iloc[0]]
        if not after_mna_date.empty:
            baseline_index = after_mna_date.index[0]
        else:
            # If no date is after MnAAnnDate, fallback to using the latest date
            baseline_index = group.index[-1]
    
    # Calculate the lag by subtracting the baseline_index from the current index
    group['lag'] = group.index - baseline_index
    
    return group


project_id = "starfire-410116"
dataset_id = "derived"
table_id = "compu_pb_stck_prc_v3" ## last version _v2 on Sep 4. Do not override!!


# Construct a reference to the target table
table_ref = client.dataset(dataset_id).table(table_id)

# Configure the load job to append data to the existing table
job_config = bigquery.LoadJobConfig(
    write_disposition=bigquery.WriteDisposition.WRITE_APPEND
)


## Iterate over df_deals to get stock price data for days around acquisition

# Suppress warnings -- reinstated at the end of the loop
warnings.filterwarnings("ignore")

# Ensure 'TR.MnAAnnDate' is in datetime format
df['mna_eventdate'] = pd.to_datetime(df['mna_eventdate'])

# Initialize an empty DataFrame to store the results
stock_prices_df = pd.DataFrame()

# Counter to track the number of lines processed
counter = 0

print(f"Loop starts...")

for index, row in df[df['mna_investorcikcode'].notna()  & (df['mna_eventdate'].notna()) ].iterrows():

    cikcode = row['mna_investorcikcode']
    mna_eventdate = row['mna_eventdate']
    mnapbdealid = row['mna_dealid']
    
    # Calculate start and end dates (mnaanndate - 7 days and mnaanndate + 7 days)
    start_date = (mna_eventdate - timedelta(days=7)).strftime('%Y-%m-%d')
    end_date = (mna_eventdate + timedelta(days=7)).strftime('%Y-%m-%d')

    print(f'-----{counter}. Downloading data for {cikcode}...')
    stock_ciks_na = conn.raw_sql(
        f"""
        SELECT
            gvkey, conm, cusip,
            exchg, fic, tpci, iid,
            datadate, cik, prccd, prcstd, cshoc, cshtrd, curcdd
        FROM comp_na_daily_all.secd
        WHERE 
            cik = '{cikcode}'
            AND '{start_date}' < datadate AND datadate < '{end_date}'
        """
    )

    stock_ciks_na = pd.DataFrame(stock_ciks_na)
    
    # Add additional columns to the result (start, end, MnAAnnDate, RIC)
    stock_ciks_na['Start'] = start_date
    stock_ciks_na['End'] = end_date
    stock_ciks_na['mna_investorcikcode'] = cikcode
    stock_ciks_na['mna_eventdate'] = mna_eventdate
    stock_ciks_na['mna_dealid'] = mnapbdealid
    stock_ciks_na['loop_counter'] = counter
    stock_ciks_na = stock_ciks_na.groupby('mna_dealid').apply(calculate_lag).reset_index(drop=True)

    # print(f'----------stock_ciks_na Shape: {stock_ciks_na.shape}')


    # Convert to date format to later process lag functions
    stock_ciks_na['datadate'] = pd.to_datetime(stock_ciks_na['datadate'], errors='coerce')
    stock_ciks_na['Start'] = pd.to_datetime(stock_ciks_na['Start'], errors='coerce')
    stock_ciks_na['End'] = pd.to_datetime(stock_ciks_na['End'], errors='coerce')

    # Append the result to the historical_data DataFrame
    stock_prices_df = pd.concat([stock_prices_df, stock_ciks_na], ignore_index=True)
    print(f'----------stock_prices_df Shape: {stock_prices_df.shape}')
    
    counter += 1

    # Print progress every 100 lines
    if counter % 1000 == 0:
        print(f'\n\n\n-----Processed {counter} lines')

        print(f'----------Appending chunk to BQ...')
        # Load the DataFrame into BigQuery
        load_job = client.load_table_from_dataframe(stock_prices_df, table_ref, job_config=job_config)
        # Wait for the load job to complete
        load_job.result()
        print(f'----------Data appended successfully\n\n\n')

        # Reinitialize master_df to not store duplicate info next time this runs
        stock_prices_df = pd.DataFrame()


## Append last chunk of code that most likely did not get into the if clause since when finished counter % 1000 != 0
print(f'\n\n\n-----Processed {counter} lines')

print(f'----------Appending chunk to BQ...')
# Load the DataFrame into BigQuery
load_job = client.load_table_from_dataframe(stock_prices_df, table_ref, job_config=job_config)
# Wait for the load job to complete
load_job.result()
print(f'----------Data appended successfully\n\n\n')

# Restore warnings
warnings.filterwarnings("default")

# Display the final historical_data DataFrame
print(stock_prices_df)


Loop starts...
-----0. Downloading data for 0001001290...
----------stock_prices_df Shape: (0, 20)
-----1. Downloading data for 0001496780...
----------stock_prices_df Shape: (0, 20)
-----2. Downloading data for 0001001290...
----------stock_prices_df Shape: (0, 20)
-----3. Downloading data for 0001875609...
----------stock_prices_df Shape: (0, 20)
-----4. Downloading data for 0001099590...
----------stock_prices_df Shape: (0, 20)
-----5. Downloading data for 0001474968...
----------stock_prices_df Shape: (0, 20)
-----6. Downloading data for 0001474968...
----------stock_prices_df Shape: (0, 20)
-----7. Downloading data for 0001525028...
----------stock_prices_df Shape: (0, 20)
-----8. Downloading data for 0001550903...
----------stock_prices_df Shape: (0, 20)
-----9. Downloading data for 0001525290...
----------stock_prices_df Shape: (0, 20)
-----10. Downloading data for 0001709135...
----------stock_prices_df Shape: (0, 20)
-----11. Downloading data for 0001804469...
----------stock_

KeyboardInterrupt: 

## Download stock price table from BQ, clean, add lags, stock index response

In [36]:
## Download data from BQ, Startups with MnA deals

print('downloading pitchbook data... \n')

# Define project on BQ
project_id = "starfire-410116"

# Authenticate to Google Cloud
client = bigquery.Client(project=project_id)

# Set allow_large_results to Truepoti
job_config = bigquery.QueryJobConfig()
job_config.allow_large_results = True

query_txt = """
SELECT 
*
FROM `starfire-410116.derived.compu_pb_stck_prc_v2` 
""" 

# Construct the SQL query to fetch data from BigQuery
query = f'{query_txt}'

# Fetch data from BigQuery
query_job = client.query(query, job_config=job_config)
results = query_job.result()  # Waits for the query to complete

# Convert the result to a pandas DataFrame
df = results.to_dataframe()

print(df.columns)
df

downloading pitchbook data... 

Index(['gvkey', 'conm', 'cusip', 'exchg', 'fic', 'tpci', 'iid', 'datadate',
       'cik', 'prccd', 'prcstd', 'cshoc', 'cshtrd', 'curcdd', 'Start', 'End',
       'mna_investorcikcode', 'mna_eventdate', 'mna_dealid', 'loop_counter'],
      dtype='object')


,gvkey,conm,cusip,exchg,fic,tpci,iid,datadate,cik,prccd,prcstd,cshoc,cshtrd,curcdd,Start,End,mna_investorcikcode,mna_eventdate,mna_dealid,loop_counter
0,005597,HERSHEY CO,427866306,1,USA,0,02,2019-08-22,0000047111,NaN,NaN,NaN,NaN,None,2019-08-21,2019-09-04,0000047111,2019-08-28,122685-85T,6946
1,007938,NACCO INDUSTRIES -CL A,629579202,1,USA,0,02,2006-08-22,0000789933,105.00,4.0,1610000.0,0.0,USD,2006-08-21,2006-09-04,0000789933,2006-08-28,20423-53T,6974
2,007938,NACCO INDUSTRIES -CL A,629579202,1,USA,0,02,2006-08-23,0000789933,105.00,4.0,1610000.0,0.0,USD,2006-08-21,2006-09-04,0000789933,2006-08-28,20423-53T,6974
3,007938,NACCO INDUSTRIES -CL A,629579202,1,USA,0,02,2006-08-24,0000789933,105.00,4.0,1610000.0,0.0,USD,2006-08-21,2006-09-04,0000789933,2006-08-28,20423-53T,6974
4,007938,NACCO INDUSTRIES -CL A,629579202,1,USA,0,02,2006-08-25,0000789933,105.00,4.0,1610000.0,0.0,USD,2006-08-21,2006-09-04,0000789933,2006-08-28,20423-53T,6974
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124022,184500,CBOE GLOBAL MARKETS INC,12503M108,21,USA,0,01,2015-06-02,0001374310,57.35,3.0,83473000.0,465655.0,USD,2015-05-26,2015-06-09,0001374310,2015-06-02,51221-26T,17826
124023,184500,CBOE GLOBAL MARKETS INC,12503M108,21,USA,0,01,2015-06-03,0001374310,57.42,3.0,83473000.0,618912.0,USD,2015-05-26,2015-06-09,0001374310,2015-06-02,51221-26T,17826
124024,184500,CBOE GLOBAL MARKETS INC,12503M108,21,USA,0,01,2015-06-04,0001374310,57.47,3.0,83473000.0,448004.0,USD,2015-05-26,2015-06-09,0001374310,2015-06-02,51221-26T,17826
124025,184500,CBOE GLOBAL MARKETS INC,12503M108,21,USA,0,01,2015-06-05,0001374310,57.10,3.0,83473000.0,384901.0,USD,2015-05-26,2015-06-09,0001374310,2015-06-02,51221-26T,17826


In [37]:
## Define fuction to create consecutive lags grouping by a given id variable
def calculate_lag(group):
    # Find the index of the row where Date equals MnAAnnDate
    matching_indices = group.index[group['datadate'] == group['mna_eventdate']].tolist()
     
    if matching_indices:
        # Exact match found
        baseline_index = matching_indices[0]
    else:
        # No exact match, find the first date after MnAAnnDate
        after_mna_date = group[group['datadate'] > group['mna_eventdate'].iloc[0]]
        if not after_mna_date.empty:
            baseline_index = after_mna_date.index[0]
        else:
            # If no date is after MnAAnnDate, fallback to using the latest date
            baseline_index = group.index[-1]
    
    # Calculate the lag by subtracting the baseline_index from the current index
    group['lag'] = group.index - baseline_index
    
    return group

## Create id variable to compute lags. Note that the loop_counter, which identifies every stock for which stock prices have
## been retrieved, returns some time different stock ids for a given gvkey. Thus, aggregate by loop_counter + iid 

df['lag_id'] = df['loop_counter'].astype(str) + "-" + df['iid']

## Compute lags
df = df.groupby('lag_id').apply(calculate_lag).reset_index(drop=True)
df






/var/tmp/ipykernel_1919/1796331257.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('lag_id').apply(calculate_lag).reset_index(drop=True)


,gvkey,conm,cusip,exchg,fic,tpci,iid,datadate,cik,prccd,...,cshtrd,curcdd,Start,End,mna_investorcikcode,mna_eventdate,mna_dealid,loop_counter,lag_id,lag
0,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2018-12-26,0001001290,217.90,...,127005.0,USD,2018-12-25,2019-01-08,0001001290,2019-01-01,153612-37T,0,0-01,-4
1,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2018-12-27,0001001290,219.68,...,210920.0,USD,2018-12-25,2019-01-08,0001001290,2019-01-01,153612-37T,0,0-01,-3
2,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2018-12-28,0001001290,221.77,...,144274.0,USD,2018-12-25,2019-01-08,0001001290,2019-01-01,153612-37T,0,0-01,-2
3,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2018-12-31,0001001290,221.67,...,82112.0,USD,2018-12-25,2019-01-08,0001001290,2019-01-01,153612-37T,0,0-01,-1
4,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2019-01-02,0001001290,226.45,...,157642.0,USD,2018-12-25,2019-01-08,0001001290,2019-01-01,153612-37T,0,0-01,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
124022,018947,VERACYTE INC,92337F107,14,USA,0,01,2021-02-03,0001384101,73.79,...,1195182.0,USD,2021-01-27,2021-02-10,0001384101,2021-02-03,166187-62T,9999,9999-01,0
124023,018947,VERACYTE INC,92337F107,14,USA,0,01,2021-02-04,0001384101,77.87,...,1141748.0,USD,2021-01-27,2021-02-10,0001384101,2021-02-03,166187-62T,9999,9999-01,1
124024,018947,VERACYTE INC,92337F107,14,USA,0,01,2021-02-05,0001384101,73.04,...,2986768.0,USD,2021-01-27,2021-02-10,0001384101,2021-02-03,166187-62T,9999,9999-01,2
124025,018947,VERACYTE INC,92337F107,14,USA,0,01,2021-02-08,0001384101,77.27,...,2621021.0,USD,2021-01-27,2021-02-10,0001384101,2021-02-03,166187-62T,9999,9999-01,3


In [38]:
df = df.sort_values(by=['lag_id', 'lag'], ascending=True)
df[['loop_counter','iid', 'lag_id', 'datadate','mna_eventdate', 'lag']].head(40)


,loop_counter,iid,lag_id,datadate,mna_eventdate,lag
0,0,01,0-01,2018-12-26,2019-01-01,-4
1,0,01,0-01,2018-12-27,2019-01-01,-3
2,0,01,0-01,2018-12-28,2019-01-01,-2
3,0,01,0-01,2018-12-31,2019-01-01,-1
4,0,01,0-01,2019-01-02,2019-01-01,0
5,0,01,0-01,2019-01-03,2019-01-01,1
6,0,01,0-01,2019-01-04,2019-01-01,2
7,0,01,0-01,2019-01-07,2019-01-01,3
8,10,90,10-90,2021-08-11,2021-08-17,-4
9,10,90,10-90,2021-08-12,2021-08-17,-3


In [45]:
project_id = "starfire-410116"
dataset_id = "derived"
table_id = "pb_deals_compu_stock_prices_v1"

# Initialize a BigQuery client
client = bigquery.Client()

# Set dataset and table reference
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig(
    autodetect=True,  # Enable auto-detection of the schema
    # source_format=bigquery.SourceFormat.CSV,
    # field_delimiter='\t',  # Specify the field delimiter as tab
    # allow_quoted_newlines=True,
    # allow_jagged_rows=True,
    max_bad_records=10000, 
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # You can choose your write disposition
    )

# Upload the DataFrame to BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()
# Wait for the job to complete


LoadJob<project=starfire-410116, location=US, id=e48dd1ca-e250-4b64-acd3-586b46a2941a>

In [46]:
# query to bring stock index prices
# Only 522 out of 124671 (0.4%) with index_closeprice null (df[df['index_closeprice'].isnull()]
# df[df['is_correct_index'].isnull()]
# df[df['is_correct_index'] == 1]
# df[df['is_correct_index'] == 0]
# df[df['fic_2'].isnull()]
# df[df['fic_2'] != 'USA'][['fic', 'fic_2']]
# len(df[df['fic'] != 'USA']) - len(df[df['fic_2'] != 'USA'])
# df[df['index_closeprice'].isnull()]



print('downloading data... \n')

# Define project on BQ
project_id = "starfire-410116"

# Authenticate to Google Cloud
client = bigquery.Client(project=project_id)

# Set allow_large_results to Truepoti
job_config = bigquery.QueryJobConfig()
job_config.allow_large_results = True

query_txt = """
WITH match AS (
SELECT
  t1.*,
  CASE WHEN t2.ticker IS NOT NULL THEN 1 ELSE 0 END AS is_correct_index,
  CASE WHEN t2.ticker IS NOT NULL THEN t1.fic ELSE 'USA' END AS fic_2,
FROM `starfire-410116.derived.pb_deals_compu_stock_prices_v1`t1
LEFT JOIN (
  SELECT
    DISTINCT * -- makes sure we drop duplicates
  FROM `starfire-410116.derived.equity_indices_all_times`
) t2
ON t1.fic = t2.`alpha-3`
  AND DATE(t1.datadate) = PARSE_DATE('%Y-%m-%d', SUBSTR(t2.date, 0 ,10))
)
SELECT
    match.*,
    t3.closeprice as index_closeprice,
    t3.date as index_date,
    t3.`major index of reference` as index_name,
    t3.ticker as index_ticker,
    t3.`sub-region` as index_sub_region,
    t3.region as index_region,
FROM match 
LEFT JOIN (
  SELECT
    DISTINCT * --makes sure we drop duplicates
  FROM `starfire-410116.derived.equity_indices_all_times`
) t3
ON match.fic_2 = t3.`alpha-3`
  AND DATE(match.datadate) = PARSE_DATE('%Y-%m-%d', SUBSTR(t3.date, 0 ,10))
""" 

# Construct the SQL query to fetch data from BigQuery
query = f'{query_txt}'

# Fetch data from BigQuery
query_job = client.query(query, job_config=job_config)
results = query_job.result()  # Waits for the query to complete

# Convert the result to a pandas DataFrame
df = results.to_dataframe()

print(df.columns)
df



downloading data... 

Index(['gvkey', 'conm', 'cusip', 'exchg', 'fic', 'tpci', 'iid', 'datadate',
       'cik', 'prccd', 'prcstd', 'cshoc', 'cshtrd', 'curcdd', 'Start', 'End',
       'mna_investorcikcode', 'mna_eventdate', 'mna_dealid', 'loop_counter',
       'lag_id', 'lag', 'is_correct_index', 'fic_2', 'index_closeprice',
       'index_date', 'index_name', 'index_ticker', 'index_sub_region',
       'index_region'],
      dtype='object')


,gvkey,conm,cusip,exchg,fic,tpci,iid,datadate,cik,prccd,...,lag_id,lag,is_correct_index,fic_2,index_closeprice,index_date,index_name,index_ticker,index_sub_region,index_region
0,061356,NOVANTA INC,67000B104,1,CAN,0,01C,2004-05-14 00:00:00+00:00,0001076930,17.5000,...,18574-01C,0,1,CAN,5904.723444,2004-05-14T00:00:00,S&P/TSX Composite index,.GSPTSE,Northern America,Americas
1,002137,BCE INC,05534B760,7,CAN,0,01C,2016-08-05 00:00:00+00:00,0000718940,62.7800,...,1249-01C,-1,1,CAN,11126.211454,2016-08-05T00:00:00,S&P/TSX Composite index,.GSPTSE,Northern America,Americas
2,002137,BCE INC,05534B851,7,CAN,1,02C,2016-08-05 00:00:00+00:00,0000718940,14.3100,...,1249-02C,-1,1,CAN,11126.211454,2016-08-05T00:00:00,S&P/TSX Composite index,.GSPTSE,Northern America,Americas
3,002137,BCE INC,05534B869,7,CAN,1,03C,2016-08-05 00:00:00+00:00,0000718940,14.2000,...,1249-03C,-1,1,CAN,11126.211454,2016-08-05T00:00:00,S&P/TSX Composite index,.GSPTSE,Northern America,Americas
4,002137,BCE INC,05534B703,7,CAN,1,06C,2016-08-05 00:00:00+00:00,0000718940,18.3500,...,1249-06C,-1,1,CAN,11126.211454,2016-08-05T00:00:00,S&P/TSX Composite index,.GSPTSE,Northern America,Americas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126002,294294,RIB SOFTWARE SE,74965G102,19,DEU,F,90,2019-10-31 00:00:00+00:00,0001588525,51.8110,...,2386-90,3,1,DEU,14346.486631,2019-10-31T00:00:00,DAX PERFORMANCE-INDEX,.GDAXI,Western Europe,Europe
126003,317891,WORLDLINE,F9867T103,19,FRA,0,01,2022-10-25 00:00:00+00:00,0001665531,43.6320,...,1754-01,0,1,FRA,6228.066679,2022-10-25T00:00:00,CAC 40 (FCHI),.FCHI,Western Europe,Europe
126004,317891,WORLDLINE,98161H101,19,FRA,F,90,2022-10-25 00:00:00+00:00,0001665531,21.7940,...,1754-90,0,1,FRA,6228.066679,2022-10-25T00:00:00,CAC 40 (FCHI),.FCHI,Western Europe,Europe
126005,006557,LAIDLAW INTERNATIONAL INC,50730K503,20,USA,0,03,1998-09-30 00:00:00+00:00,0000737874,9.4375,...,6158-03,-1,1,USA,1017.010000,1998-09-30T00:00:00,S&P500,.SP500,Northern America,Americas


In [47]:
df = df.sort_values(by=['lag_id', 'lag'], ascending=True)
df

,gvkey,conm,cusip,exchg,fic,tpci,iid,datadate,cik,prccd,...,lag_id,lag,is_correct_index,fic_2,index_closeprice,index_date,index_name,index_ticker,index_sub_region,index_region
118931,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2018-12-26 00:00:00+00:00,0001001290,217.90,...,0-01,-4,1,BMU,295.3000,2018-12-26T00:00:00,Refinitiv Latin America,.TRXFLDLAPU,Northern America,Americas
74499,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2018-12-27 00:00:00+00:00,0001001290,219.68,...,0-01,-3,1,BMU,298.7800,2018-12-27T00:00:00,Refinitiv Latin America,.TRXFLDLAPU,Northern America,Americas
43704,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2018-12-28 00:00:00+00:00,0001001290,221.77,...,0-01,-2,1,BMU,304.5100,2018-12-28T00:00:00,Refinitiv Latin America,.TRXFLDLAPU,Northern America,Americas
111457,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2018-12-31 00:00:00+00:00,0001001290,221.67,...,0-01,-1,1,BMU,304.9000,2018-12-31T00:00:00,Refinitiv Latin America,.TRXFLDLAPU,Northern America,Americas
59022,061452,CREDICORP LTD,G2519Y108,11,BMU,0,01,2019-01-02 00:00:00+00:00,0001001290,226.45,...,0-01,0,1,BMU,318.4200,2019-01-02T00:00:00,Refinitiv Latin America,.TRXFLDLAPU,Northern America,Americas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123951,018947,VERACYTE INC,92337F107,14,USA,0,01,2021-02-03 00:00:00+00:00,0001384101,73.79,...,9999-01,0,1,USA,3830.1725,2021-02-03T00:00:00,S&P500,.SP500,Northern America,Americas
117934,018947,VERACYTE INC,92337F107,14,USA,0,01,2021-02-04 00:00:00+00:00,0001384101,77.87,...,9999-01,1,1,USA,3871.7406,2021-02-04T00:00:00,S&P500,.SP500,Northern America,Americas
19052,018947,VERACYTE INC,92337F107,14,USA,0,01,2021-02-05 00:00:00+00:00,0001384101,73.04,...,9999-01,2,1,USA,3886.8317,2021-02-05T00:00:00,S&P500,.SP500,Northern America,Americas
106915,018947,VERACYTE INC,92337F107,14,USA,0,01,2021-02-08 00:00:00+00:00,0001384101,77.27,...,9999-01,3,1,USA,3915.5886,2021-02-08T00:00:00,S&P500,.SP500,Northern America,Americas


In [48]:
project_id = "starfire-410116"
dataset_id = "derived"
table_id = "pb_deals_compu_stock_prices_v1"

# Initialize a BigQuery client
client = bigquery.Client()

# Set dataset and table reference
dataset_ref = client.dataset(dataset_id)
table_ref = dataset_ref.table(table_id)

job_config = bigquery.LoadJobConfig(
    autodetect=True,  # Enable auto-detection of the schema
    # source_format=bigquery.SourceFormat.CSV,
    # field_delimiter='\t',  # Specify the field delimiter as tab
    # allow_quoted_newlines=True,
    # allow_jagged_rows=True,
    max_bad_records=10000, 
    write_disposition=bigquery.WriteDisposition.WRITE_TRUNCATE  # You can choose your write disposition
    )

# Upload the DataFrame to BigQuery
job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
job.result()
# Wait for the job to complete


LoadJob<project=starfire-410116, location=US, id=112b045b-7a68-4f30-9692-773145fe5bb7>